'GetStore: Internal Server Error, Domain: CoreRuntime, Severity: Error, errorId: 2000, Service operation GetStore is unknown'
{'errorMessage': {'error': {'errorId': '2000', 'domain': 'CoreRuntime', 'severity': 'Error', 'category': 'Request', 'message': 'Service operation GetStore is unknown', 'subdomain': 'Inbound_Meta_Data', 'parameter': {'_name': 'Param1', 'value': 'GetStore'}}}}


NameError: name 'client_id' is not defined